In [22]:
import os
import torch
import numpy as np
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

from torch.utils.data import TensorDataset, DataLoader, Dataset

from utils.util import *

In [2]:
DATA_PATH = '/home/ralleking/Code/Python/Datasets/meastro'
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

## pre-processing

In [3]:
folds = read_txt_files(DATA_PATH)

In [156]:
train = np.array(folds[2])
train_labels = np.array(folds[3])

#trying to predict only next note, fix column order
train_labels = train_labels[:,:1]
train = np.transpose(train, (0,2,1))

train_dataset = TensorDataset(torch.from_numpy(train).float().to(DEVICE), torch.from_numpy(train_labels).float().to(DEVICE))

## model
### predicts only the next timestep, based on 15x100 input size

In [158]:
class Baseline(nn.Module):
    """
    Simple baseline lstm model
    """

    def __init__(self, time_steps, hidden_dim, num_classes):

        super(Baseline, self).__init__()
        self.hidden_dim = hidden_dim

        #lstm layer
        self.lstm = nn.LSTM(time_steps, hidden_dim)

        #output layer
        self.linear = nn.Linear(hidden_dim, num_classes)

        #dropout
        self.do = nn.Dropout(0.6)

    def forward(self, input):
        lstm_out, _ = self.lstm(input.view(len(input), 1, -1))
        out = self.do(lstm_out)
        predictions = self.linear(out.view(len(input), -1))
        return predictions[-1]


## main

In [159]:
time_steps = 100
num_classes = 15

In [160]:
#model definitions
model = Baseline(time_steps, 12, num_classes)
model.to(DEVICE)
loss_function = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

epochs = 10

for i in range(epochs):
    for x, y in train_dataset:
        optimizer.zero_grad()
        y_pred = model(x)
        single_loss = loss_function(y_pred, y)
        single_loss.backward()
        optimizer.step()
    print(single_loss)

tensor(191.2746, device='cuda:0', grad_fn=<MseLossBackward>)
tensor(61.3971, device='cuda:0', grad_fn=<MseLossBackward>)
tensor(59.1782, device='cuda:0', grad_fn=<MseLossBackward>)
tensor(56.9319, device='cuda:0', grad_fn=<MseLossBackward>)
tensor(72.8357, device='cuda:0', grad_fn=<MseLossBackward>)
tensor(53.1427, device='cuda:0', grad_fn=<MseLossBackward>)
tensor(55.4849, device='cuda:0', grad_fn=<MseLossBackward>)
tensor(54.2258, device='cuda:0', grad_fn=<MseLossBackward>)
tensor(55.8322, device='cuda:0', grad_fn=<MseLossBackward>)
tensor(51.6202, device='cuda:0', grad_fn=<MseLossBackward>)


In [182]:
pred = model(train_dataset[0][0])
pred

tensor([ 6.4623e+01,  3.4164e+01,  4.5388e+01,  1.9326e-01,  4.7245e-02,
         8.0082e-02,  1.0565e-01,  2.0081e-02,  1.3719e-01,  7.8275e-03,
         1.5141e-01,  1.1283e-01, -4.6992e-02,  1.6119e-01,  1.1325e-02],
       device='cuda:0', grad_fn=<SelectBackward>)